<a href="https://colab.research.google.com/github/andres-merino/AprendizajeAutomaticoInicial-05-N0105/blob/main/2-Ejercicios/10-Optimizacion-Hiperparametros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table style="border: none; border-collapse: collapse;">
    <tr>
        <td style="width: 20%; vertical-align: middle; padding-right: 10px;">
            <img src="https://i.imgur.com/nt7hloA.png" width="100">
        </td>
        <td style="width: 2px; text-align: center;">
            <font color="#0030A1" size="7">|</font><br>
            <font color="#0030A1" size="7">|</font>
        </td>
        <td>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="5">
                <b>Escuela de Ciencias Físicas y Matemática</b>
            </font> </p>
            <p style="font-variant: small-caps;"><font color="#0030A1" size="4">
                Aprendizaje Automático Inicial &bull; Optmización de Hiperparámetros
            </font></p>
            <p style="font-style: oblique;"><font color="#0030A1" size="3">
                Andrés Merino &bull; 2024-02
            </font></p>
        </td>  
    </tr>
</table>

---
## <font color='264CC7'> Introducción </font>

A lo largo de este taller, aplicaremos optimización de hiperparámetros en un modelo que elijas.

Los paquetes necesarios son:

In [32]:
# Paquetes necesarios
import pandas as pd  # Manejo de datos
import matplotlib.pyplot as plt  # Visualización

from sklearn.model_selection import train_test_split # División de datos
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Métrica de evaluación

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, KFold  # Búsqueda de hiperparámetros
from sklearn.ensemble import RandomForestClassifier

import joblib
import pickle

---
## <font color='264CC7'> Clasificación </font>


### <font color='264CC7'> Preprocesamiento de datos </font>

Primero necesitas el conjunto de datos. Los datos a utilzar son los seleccionados en la clase anterior.

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Carga el conjunto de datos y procésalos:
<ul>
  <li>Muestra algunos datos.</li>
  <li>Muestra una descripción de los datos.</li>
  <li>Escala los datos si es necesario.</li>
</ul>
</div>

In [3]:
# Cargar los datos
url = "https://raw.githubusercontent.com/martinbacaia/datasets/refs/heads/main/Cancer_Data.csv"
df = pd.read_csv(url)

# Eliminar la columna no deseada
df = df.drop(columns=['Unnamed: 32'])

# Convertir 'diagnosis' a valores binarios
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

# Separar características y etiquetas
X = df.drop(columns=['id', 'diagnosis'])
y = df['diagnosis']

df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
df.describe()


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,0.372583,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,0.483918,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,0.000000,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,0.000000,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,0.000000,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,1.000000,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,1.000000,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500



### <font color='264CC7'> Modelo </font>


<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Selecciona el mejor modelo de las clases anteriores.
<ul>
  <li>Muestra los hiperparámetros del modelo.</li>
  <li>Consulta qué significan al menos 4 hiperparámetros.</li>
  <li>Selecciona los hiperparámetros que deseas optimizar, al menos 3.</li>
</ul>
</div>

In [9]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de Random Forest
model = RandomForestClassifier(random_state=42)

# Parámetros del modelo
model.get_params()


{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

####**Hiperparámetros del modelo**
1. n_estimators: 100
Número de árboles en el bosque. Aumentar este valor generalmente mejora el rendimiento pero incrementa el tiempo de entrenamiento.

2. max_depth: None
Profundidad máxima de cada árbol en el bosque. Controla la complejidad del modelo y ayuda a prevenir el sobreajuste.

3. criterion: 'gini'
Función de calidad para medir la división. En este caso, se utiliza el índice de Gini para evaluar la pureza de los nodos.

4. bootstrap: True
Indica si se utiliza muestreo con reemplazo para entrenar los árboles individuales. Es una característica clave de los Random Forests.

####**Hiperparámetros seleccionados para optimización**
1. n_estimators: Evaluar el efecto de diferentes números de árboles, por ejemplo, [100, 200, 300].
2. max_depth: Ajustar la profundidad máxima para evitar sobreajuste, por ejemplo, [5, 10, 20, None].
3. max_features: Determinar cuántas características considerar en cada división, por ejemplo, ['auto', 'sqrt', 'log2'].

### <font color='264CC7'> Optimización por GridSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica GridSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 3 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [15]:
parametros = {'n_estimators': [100, 150, 200],
              'max_depth': [5, 10, 20],
              'max_leaf_nodes': [10, 20, 50]}
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

modelo = GridSearchCV(model, parametros, cv=k_fold, scoring='accuracy')
modelo

GridSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [5, 10, 20],
                         'max_leaf_nodes': [10, 20, 50],
                         'n_estimators': [100, 150, 200]},
             scoring='accuracy')

In [16]:
# Definir los k-folds
modelo.fit(X_train, y_train)

# Mostrar los resultados
pd.DataFrame(modelo.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_max_leaf_nodes,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.219368,0.008895,0.009552,0.000635,5,10,100,"{'max_depth': 5, 'max_leaf_nodes': 10, 'n_esti...",0.945055,0.967033,0.989011,0.912088,0.923077,0.947253,0.028146,23
1,0.322009,0.019298,0.016779,0.004919,5,10,150,"{'max_depth': 5, 'max_leaf_nodes': 10, 'n_esti...",0.956044,0.967033,0.989011,0.912088,0.934066,0.951648,0.026556,13
2,0.671606,0.015578,0.026362,0.001114,5,10,200,"{'max_depth': 5, 'max_leaf_nodes': 10, 'n_esti...",0.956044,0.967033,0.989011,0.912088,0.934066,0.951648,0.026556,13
3,0.218360,0.006946,0.009690,0.000166,5,20,100,"{'max_depth': 5, 'max_leaf_nodes': 20, 'n_esti...",0.934066,0.967033,0.978022,0.901099,0.934066,0.942857,0.027274,26
4,0.326577,0.010839,0.012655,0.000332,5,20,150,"{'max_depth': 5, 'max_leaf_nodes': 20, 'n_esti...",0.956044,0.967033,0.989011,0.923077,0.934066,0.953846,0.023466,7
5,0.431490,0.004400,0.017058,0.001814,5,20,200,"{'max_depth': 5, 'max_leaf_nodes': 20, 'n_esti...",0.956044,0.967033,1.000000,0.923077,0.934066,0.956044,0.026917,1
6,0.220215,0.005106,0.009284,0.000209,5,50,100,"{'max_depth': 5, 'max_leaf_nodes': 50, 'n_esti...",0.934066,0.967033,0.978022,0.901099,0.934066,0.942857,0.027274,26
7,0.322007,0.010305,0.014222,0.001371,5,50,150,"{'max_depth': 5, 'max_leaf_nodes': 50, 'n_esti...",0.956044,0.967033,0.989011,0.923077,0.934066,0.953846,0.023466,7
8,0.463640,0.060583,0.018160,0.003643,5,50,200,"{'max_depth': 5, 'max_leaf_nodes': 50, 'n_esti...",0.956044,0.967033,1.000000,0.923077,0.934066,0.956044,0.026917,1
9,0.343768,0.014168,0.015712,0.002198,10,10,100,"{'max_depth': 10, 'max_leaf_nodes': 10, 'n_est...",0.945055,0.967033,0.978022,0.912088,0.923077,0.945055,0.025059,24


In [17]:
# Mejores parámetros
print("Mejores parámetros", modelo.best_params_)
print("Mejor score", modelo.best_score_)

Mejores parámetros {'max_depth': 5, 'max_leaf_nodes': 20, 'n_estimators': 200}
Mejor score 0.9560439560439562


In [19]:
# Realizar predicciones y evaluar el modelo
y_pred = modelo.predict(X_test)

# Precisión del modelo con dos decimales
accuracy = round(accuracy_score(y_test, y_pred), 2)
print("Precisión del modelo:", accuracy)

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(cm)

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

Precisión del modelo: 0.96
Matriz de confusión:
[[70  1]
 [ 3 40]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97        71
           1       0.98      0.93      0.95        43

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



### <font color='264CC7'> Optimización por RandomSearch </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
    Aplica RandomSearch para optimizar los hiperparámetros del modelo.
<ul>
  <li>Para cada hiperparámetro, selecciona al menos 5 valores, si es posible.</li>
  <li>Utiliza al menos 5 validaciones cruzadas.</li>
  <li>Usa RandomSearchCV con 25 iteraciones.</li>
  <li>Muestra los parámetros óptimos y su score.</li>
</ul>
</div>

In [27]:
# Hiperparámetros y sus rangos
parametros = {
    'n_estimators': [50, 100, 150, 200, 250],
    'max_depth': [5, 10, 20, 30, None],
    'max_leaf_nodes': [10, 20, 50, 100, None],
}

# Configuración de k-fold
k_fold = KFold(n_splits=5, shuffle=True, random_state=42)

# RandomizedSearchCV con 25 iteraciones
modelo = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=parametros,
    n_iter=25,  # Iteraciones
    scoring='accuracy',
    cv=k_fold,  # Validación cruzada
    random_state=42,
    n_jobs=-1   # Paraleliza en todos los núcleos disponibles
)

modelo

RandomizedSearchCV(cv=KFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=RandomForestClassifier(random_state=42), n_iter=25,
                   n_jobs=-1,
                   param_distributions={'max_depth': [5, 10, 20, 30, None],
                                        'max_leaf_nodes': [10, 20, 50, 100,
                                                           None],
                                        'n_estimators': [50, 100, 150, 200,
                                                         250]},
                   random_state=42, scoring='accuracy')

In [28]:
# Ajustar el modelo
modelo.fit(X_train, y_train)

# Mostrar los resultados
pd.DataFrame(modelo.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_leaf_nodes,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.130597,0.168321,0.051364,0.014470,200,100,5,"{'n_estimators': 200, 'max_leaf_nodes': 100, '...",0.956044,0.967033,1.000000,0.923077,0.934066,0.956044,0.026917,1
1,0.983236,0.139643,0.038762,0.008099,150,100,10,"{'n_estimators': 150, 'max_leaf_nodes': 100, '...",0.956044,0.967033,0.989011,0.923077,0.945055,0.956044,0.021978,1
2,0.443741,0.138779,0.019835,0.007398,100,50,10,"{'n_estimators': 100, 'max_leaf_nodes': 50, 'm...",0.956044,0.967033,0.978022,0.912088,0.945055,0.951648,0.022628,6
3,0.411052,0.067748,0.023342,0.009737,100,10,30,"{'n_estimators': 100, 'max_leaf_nodes': 10, 'm...",0.945055,0.967033,0.978022,0.912088,0.923077,0.945055,0.025059,22
4,1.175778,0.080105,0.058155,0.028677,200,10,20,"{'n_estimators': 200, 'max_leaf_nodes': 10, 'm...",0.956044,0.967033,0.989011,0.912088,0.934066,0.951648,0.026556,6
5,0.539780,0.096009,0.021505,0.006380,100,100,30,"{'n_estimators': 100, 'max_leaf_nodes': 100, '...",0.956044,0.967033,0.978022,0.912088,0.945055,0.951648,0.022628,6
6,0.648850,0.169385,0.037421,0.008927,100,20,20,"{'n_estimators': 100, 'max_leaf_nodes': 20, 'm...",0.956044,0.967033,0.978022,0.912088,0.945055,0.951648,0.022628,6
7,1.996794,0.365370,0.058763,0.027128,200,None,None,"{'n_estimators': 200, 'max_leaf_nodes': None, ...",0.956044,0.967033,0.978022,0.934066,0.923077,0.951648,0.020382,6
8,1.603014,0.579942,0.038869,0.024312,200,100,30,"{'n_estimators': 200, 'max_leaf_nodes': 100, '...",0.956044,0.967033,0.989011,0.923077,0.934066,0.953846,0.023466,4
9,0.838687,0.015839,0.033579,0.000709,250,10,5,"{'n_estimators': 250, 'max_leaf_nodes': 10, 'm...",0.956044,0.967033,0.989011,0.912088,0.923077,0.949451,0.028317,19


In [29]:
# Mejores parámetros
print("Mejores parámetros", modelo.best_params_)
print("Mejor score", modelo.best_score_)

Mejores parámetros {'n_estimators': 200, 'max_leaf_nodes': 100, 'max_depth': 5}
Mejor score 0.9560439560439562


### <font color='264CC7'> Guardado de modelo </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Con los parámetros óptimos que mejor resultado dieron, reentrena el modelo, muestra su score y guárdalo.
</div>

In [30]:
# Crear el modelo de Random Forest
model = RandomForestClassifier(random_state=42, n_estimators= 200, max_leaf_nodes= 20, max_depth= 5)

# Realizar predicciones y evaluar el modelo
y_pred = modelo.predict(X_test)

# Precisión del modelo con dos decimales
accuracy = round(accuracy_score(y_test, y_pred), 2)
print("Precisión del modelo:", accuracy)

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(cm)

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

Precisión del modelo: 0.96
Matriz de confusión:
[[70  1]
 [ 3 40]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97        71
           1       0.98      0.93      0.95        43

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



In [31]:
# Crear el modelo de Random Forest
model = RandomForestClassifier(random_state=42, n_estimators= 200, max_leaf_nodes= 100, max_depth= 5)

# Realizar predicciones y evaluar el modelo
y_pred = modelo.predict(X_test)

# Precisión del modelo con dos decimales
accuracy = round(accuracy_score(y_test, y_pred), 2)
print("Precisión del modelo:", accuracy)

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(cm)

# Reporte de clasificación
print("Reporte de clasificación:")
print(classification_report(y_test, y_pred))

Precisión del modelo: 0.96
Matriz de confusión:
[[70  1]
 [ 3 40]]
Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97        71
           1       0.98      0.93      0.95        43

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114



In [33]:
# Guardar el modelo
with open('modelo_cancer.pkl', 'wb') as file:
    pickle.dump(modelo, file)

with open('modelo_cancer.joblib', 'wb') as file:
    joblib.dump(modelo, file)

### <font color='264CC7'> Publicación </font>

<div style="background-color: #edf1f8; border-color: #264CC7; border-left: 5px solid #264CC7; padding: 0.5em;">
<strong>Ejercicio:</strong><br>
  Coloca el este cuaderno y el modelo en tu repositorio de GitHub. Agrega una licencia MIT y un README.md donde se explique el contenido del repositorio, los datos utilizados y los resultados obtenidos.
</div>